<a href="https://colab.research.google.com/github/ZzzMirror/next_day_wildfire_spread/blob/master/LSTM_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls /content/*.tfrecord
file_path = '/content/*.tfrecord'

/content/next_day_wildfire_spread_eval_00.tfrecord
/content/next_day_wildfire_spread_eval_01.tfrecord
/content/next_day_wildfire_spread_train_00.tfrecord
/content/next_day_wildfire_spread_train_01.tfrecord
/content/next_day_wildfire_spread_train_02.tfrecord
/content/next_day_wildfire_spread_train_03.tfrecord
/content/next_day_wildfire_spread_train_04.tfrecord
/content/next_day_wildfire_spread_train_05.tfrecord
/content/next_day_wildfire_spread_train_06.tfrecord
/content/next_day_wildfire_spread_train_07.tfrecord
/content/next_day_wildfire_spread_train_08.tfrecord
/content/next_day_wildfire_spread_train_09.tfrecord
/content/next_day_wildfire_spread_train_10.tfrecord
/content/next_day_wildfire_spread_train_11.tfrecord
/content/next_day_wildfire_spread_train_12.tfrecord
/content/next_day_wildfire_spread_train_13.tfrecord
/content/next_day_wildfire_spread_train_14.tfrecord


In [ ]:
import tensorflow as tf

### 定义特征

In [ ]:
# 定义特征
feature_description = {
    'vs': tf.io.VarLenFeature(tf.float32),
    'NDVI': tf.io.VarLenFeature(tf.float32),
    'erc': tf.io.VarLenFeature(tf.float32),
    'PrevFireMask': tf.io.VarLenFeature(tf.float32),
    'FireMask': tf.io.VarLenFeature(tf.float32),
    'sph': tf.io.VarLenFeature(tf.float32),
    'th': tf.io.VarLenFeature(tf.float32),
    'pr': tf.io.VarLenFeature(tf.float32),
    'population': tf.io.VarLenFeature(tf.float32),
    'tmmx': tf.io.VarLenFeature(tf.float32),
    'pdsi': tf.io.VarLenFeature(tf.float32),
    'elevation': tf.io.VarLenFeature(tf.float32),
    'tmmn': tf.io.VarLenFeature(tf.float32)
}

In [ ]:
#DATA_STATS ，包含每个特征的统计数据（最小值、最大值、均值、标准差）
DATA_STATS = {
    'elevation': (0.0, 3141.0, 657.3003, 649.0147),
    'th': (0., 360.0, 190.3298, 72.5985),
    'vs': (0.0, 10.0243, 3.8501, 1.4110),
    'tmmn': (253.15, 298.9489, 281.08768, 8.9824),
    'tmmx': (253.15, 315.0923, 295.17383, 9.8155),
    'sph': (0., 1., 0.0071658953, 0.0042835088),
    'pr': (0.0, 44.5304, 1.7398051, 4.4828),
    'pdsi': (-6.1298, 7.8760, -0.0053, 2.6823),
    'NDVI': (-9821.0, 9996.0, 5157.625, 2466.6677),
    'population': (0., 2534.0630, 25.5314, 154.7233),
    'erc': (0.0, 106.2489, 37.3263, 20.8460),
    'PrevFireMask': (-1., 1., 0., 1.)
}

### 解析函数

In [ ]:
def parse_tfrecord(serialized_example):
    example = tf.io.parse_single_example(serialized_example, feature_description)
    features = {k: tf.sparse.to_dense(v) for k, v in example.items() if k != 'FireMask'}
    label = tf.sparse.to_dense(example['FireMask'])
    return features, label

解析函数测试：

In [ ]:
def test_parse_tfrecord():
    # 创建一个模拟的 TFRecord 示例
    feature = {
        'vs': tf.train.Feature(float_list=tf.train.FloatList(value=[1.0, 2.0])),
        'NDVI': tf.train.Feature(float_list=tf.train.FloatList(value=[0.5])),
        'FireMask': tf.train.Feature(float_list=tf.train.FloatList(value=[0, 1]))
    }
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    serialized_example = example.SerializeToString()

    # 解析模拟的 TFRecord
    parsed_features, parsed_label = parse_tfrecord(serialized_example)

    # 检查解析的结果
    print("Features:", parsed_features)
    print("Label:", parsed_label)

# 定义解析函数
feature_description = {
    'vs': tf.io.VarLenFeature(tf.float32),
    'NDVI': tf.io.VarLenFeature(tf.float32),
    'FireMask': tf.io.VarLenFeature(tf.float32)
}



# 运行测试
test_parse_tfrecord()


Features: {'NDVI': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.5], dtype=float32)>, 'vs': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>}
Label: tf.Tensor([0. 1.], shape=(2,), dtype=float32)


### 加载数据集并创建数据

In [ ]:
def load_dataset(file_path):
    # 使用 tf.io.gfile.glob 解析包含通配符的路径
    files = tf.io.gfile.glob(file_path)
    if not files:
        raise FileNotFoundError("No files matched the pattern: {}".format(pattern))

    # 创建数据集
    raw_dataset = tf.data.TFRecordDataset(files)
    parsed_dataset = raw_dataset.map(parse_tfrecord)
    return parsed_dataset

# 使用通配符加载多个文件示例
train_dataset = load_dataset('next_day_wildfire_spread_train*')
val_dataset = load_dataset('next_day_wildfire_spread_eval*')

加载数据测试：

In [ ]:
def inspect_dataset_and_guess_label_key(dataset):
    for features, label in dataset.take(1):  # 只取一个批次
        print("Feature shapes:")
        all_keys = features.keys()  # 获取所有键的列表
        for key in all_keys:
            print(f"{key}: {features[key].shape}")

        # 打印标签的形状
        print("Label shape:", label.shape)

        # 猜测 label_key
        # 常用逻辑：标签字段可能是二元的、分类的，或者是区别明显的连续值
        print("Potential label_key candidates based on common naming conventions:")
        for key in all_keys:
            if "label" in key.lower() or "target" in key.lower() or "mask" in key.lower():
                print(key)

# 使用此函数查看数据集并猜测标签键
inspect_dataset_and_guess_label_key(train_dataset)


Feature shapes:
NDVI: (4096,)
PrevFireMask: (4096,)
elevation: (4096,)
erc: (4096,)
pdsi: (4096,)
population: (4096,)
pr: (4096,)
sph: (4096,)
th: (4096,)
tmmn: (4096,)
tmmx: (4096,)
vs: (4096,)
Label shape: (4096,)
Potential label_key candidates based on common naming conventions:
PrevFireMask


In [ ]:
#输出数据集特征名称和类型
import tensorflow as tf

def print_features(tfrecord_file):
    # Load one example from the TFRecord file
    raw_dataset = tf.data.TFRecordDataset(tfrecord_file)
    for raw_record in raw_dataset.take(1):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        print("Feature names and data types in the TFRecord:")
        for key, feature in example.features.feature.items():
            # Print the feature name and the data type of the feature
            data_type = feature.WhichOneof('kind')
            print(f"{key}: {data_type}")

# Replace 'path_to_tfrecord.tfrecord' with the path to your TFRecord file
print_features('/content/next_day_wildfire_spread_train_00.tfrecord')


Feature names and data types in the TFRecord:
vs: float_list
NDVI: float_list
FireMask: float_list
sph: float_list
th: float_list
pr: float_list
PrevFireMask: float_list
tmmn: float_list
elevation: float_list
tmmx: float_list
pdsi: float_list
erc: float_list
population: float_list


In [ ]:
def test_load_and_preprocess():
    # 假设您已经有一个小的 TFRecord 文件用于测试
    test_dataset = load_dataset(file_path)
    for features, label in test_dataset.take(1):  # 只取一个批次进行验证
        print("Batch features:", features)
        print("Batch labels:", label)

# 实际调用测试函数
test_load_and_preprocess()


Batch features: {'NDVI': <tf.Tensor: shape=(4096,), dtype=float32, numpy=array([5567., 5664., 5379., ..., 5162., 4169., 4169.], dtype=float32)>, 'vs': <tf.Tensor: shape=(4096,), dtype=float32, numpy=
array([3.7468874, 3.7470345, 3.7470841, ..., 3.9530666, 3.9572198,
       3.9624588], dtype=float32)>}
Batch labels: tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(4096,), dtype=float32)


### 数据预处理

In [ ]:
def preprocess(features, label):
    # 处理特征
    features['NDVI'] = (features['NDVI'] + 10000) / 20000
    features['tmmn'] -= 273.15
    features['tmmx'] -= 273.15

    for key in ['elevation', 'population', 'vs', 'sph']:
        mean = DATA_STATS[key][2]  # mean均值
        std = DATA_STATS[key][3]   # std标准差
        features[key] = (features[key] - mean) / std

    # 不需要从特征中提取标签
    # label 已经作为独立参数传递

    # 组合其它特征
    feature_values = [features[key] for key in features if key in features]
    features_tensor = tf.stack(feature_values, axis=-1)
    return features_tensor, label


In [ ]:
# 应用预处理和数据管道设置
train_dataset = train_dataset.map(preprocess).batch(32).shuffle(buffer_size=1000).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)


In [ ]:
def print_feature_names(dataset):
    for features, label in dataset.take(1):  # 只取一个批次
        # 假设 features 是一个字典
        if isinstance(features, dict):
            print("Feature names in the dataset:")
            for key in features.keys():
                print(key)
        else:
            print("Features are not provided in a dictionary format.")

# 调用此函数查看训练和验证数据集的特征名称
print_feature_names(train_dataset)


### 定义LSTM模型

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        LSTM(50),
        Dense(1, activation='sigmoid')  # 根据问题调整激活函数和输出层神经元数量
    ])
    return model

# 创建模型实例
model = build_model((None, 12))  # None 表示任意时间序列长度


In [ ]:
def build_model(input_shape):
    model = Sequential([
    LSTM(128, return_sequences=True, input_shape=input_shape),
    LSTM(128, return_sequences=True),
    LSTM(128),
    Dense(1, activation='sigmoid')
])
    return model

# 创建模型实例
model = build_model((None, 12))  # None 表示任意时间序列长度


### 编译和训练模型

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_dataset, epochs=3, validation_data=val_dataset)


Epoch 1/3


DataLossError: Graph execution error:

Detected at node IteratorGetNext defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.10/dist-pack